# ML Model in 1hr

This script will build a ML model in 1hr.  Much of the data munging and feature engineering will be left out of this model.  This is a practice script for the steps of ML modeling.

## - Data Set-up

 Importing libraries. Read test and train data. Append both.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

In [3]:
#Flag for Train and Test Data set
train_data['Type']='Train' 
test_data['Type']='Test'

In [4]:
#Combined 'Train' and 'Test' Data.
full_data = pd.concat([train_data,test_data],axis=0,sort=True) 

## - Descriptive Analysis / Data Exploration:

###### Sample of Methods

In [5]:
"""
full_data.head()
full_data.shape
full_data.columns
full_data.describe()
full_data.dtypes
"""

'\nfull_data.head()\nfull_data.shape\nfull_data.columns\nfull_data.describe()\nfull_data.dtypes\n'

In [6]:
full_data.columns

Index(['Age', 'Education', 'Hours.Per.Week', 'ID', 'Income.Group',
       'Marital.Status', 'Native.Country', 'Occupation', 'Race',
       'Relationship', 'Sex', 'Type', 'Workclass'],
      dtype='object')

In [7]:
#Box Plots
"""
plt.boxplot(full_data['Age'])
plt.ylabel('Age')
plt.title('Box Plot of Age')
plt.show()
"""

"\nplt.boxplot(full_data['Age'])\nplt.ylabel('Age')\nplt.title('Box Plot of Age')\nplt.show()\n"

In [8]:
#Histograms
"""
plt.hist(full_data['Age'],bins=8)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Frequency of Age')
plt.show()
"""

"\nplt.hist(full_data['Age'],bins=8)\nplt.xlabel('Age')\nplt.ylabel('Frequency')\nplt.title('Frequency of Age')\nplt.show()\n"

In [9]:
#full_data.plot(kind='scatter',x='Age',y='Hours.Per.Week')
#plt.show()

###### Identify ID variables, Target variables, Categorical Variables, Numerical Variables, and Other

In [10]:
ID_col = ['ID']
target_col = ["Hours.Per.Week"]
cat_cols = ['Education', 'Income.Group',
       'Marital.Status', 'Native.Country', 'Occupation', 'Race',
       'Relationship', 'Sex', 'Type', 'Workclass']
num_cols= list(set(list(full_data.columns))-set(cat_cols)-set(ID_col)-set(target_col))

In [11]:
#Test and Train Data identifier
other_col=['Type'] 

##  - Data Treatment (Missing values treatment):

In [12]:
#full_data.isnull().any()

###### Sample of Methods

In [13]:
"""
pd.isnull(full_data[‘Occupation’]) 
pd.notnull(full_data['Occupation'])
pd.isnull(full_data['Occupation']).values.ravel().sum()
pd.nottnull(full_data['Occupation']).values.ravel().sum()
full_data.dropna(axis=0,how='all')
full_data.dropna(axis=0,how='any')
full_ata.fillna(0)
full_data.fillna(‘missing’)
full_data[‘Occupation’].fillna(0)
data['body']
data['age']
data['age'].fillna(data['age'].mean())
data['age'].fillna(method='ffill')
data['age'].fillna(method='backfill')
"""


"\npd.isnull(full_data[‘Occupation’]) \npd.notnull(full_data['Occupation'])\npd.isnull(full_data['Occupation']).values.ravel().sum()\npd.nottnull(full_data['Occupation']).values.ravel().sum()\nfull_data.dropna(axis=0,how='all')\nfull_data.dropna(axis=0,how='any')\nfull_ata.fillna(0)\nfull_data.fillna(‘missing’)\nfull_data[‘Occupation’].fillna(0)\ndata['body']\ndata['age']\ndata['age'].fillna(data['age'].mean())\ndata['age'].fillna(method='ffill')\ndata['age'].fillna(method='backfill')\n"

In [14]:
"""
dummy_sex = pd.get_dummies(data['Sex'],prefix='Sex')
column_name = full_data.columns.values.tolist()
column_name.remove('Sex')
full_data[column_name].join(dummy_sex)
"""

"\ndummy_sex = pd.get_dummies(data['Sex'],prefix='Sex')\ncolumn_name = full_data.columns.values.tolist()\ncolumn_name.remove('Sex')\nfull_data[column_name].join(dummy_sex)\n"

###### Identify Missing Values and Create Flag

In [15]:
# Combined numerical and Categorical variables for missing value treatment
num_cat_cols = num_cols+cat_cols 

#Loop in a new variable for each missing variable and flag missing value with 1
for var in num_cat_cols:
    if full_data[var].isnull().any()==True:
        full_data[var+'_NA']=full_data[var].isnull()*1 

###### Impute Missing values

In [16]:
#Impute numerical missing values with mean
full_data[num_cols] = full_data[num_cols].fillna(full_data[num_cols].mean(),inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [17]:
#Impute categorical missing values with -9999
full_data[cat_cols] = full_data[cat_cols].fillna(value = -9999)

## - Data Modelling:

###### Label encoders for Categorical Variables and Split Training and Testing Data

In [18]:
#create label encoders for categorical features
for var in cat_cols:
 number = LabelEncoder()
 full_data[var] = number.fit_transform(full_data[var].astype('str'))

In [19]:
#If target variable is categorical then convert it
#full_data["N/A"] = number.fit_transform(full_data["N/A"].astype('str'))

In [20]:
#Split data
train = full_data[full_data['Type']=='Train']
test = full_data[full_data['Type']=='Test']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [21]:
train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

###### Pass Imputed and Dummy (missing values flags) Variables into Model

In [22]:
#Using random forest to predict the class
features = list(set(list(full_data.columns))-set(ID_col)-set(target_col))

In [23]:
x_train = Train[list(features)].values
y_train = Train["Hours.Per.Week"].values
x_validate = Validate[list(features)].values
y_validate = Validate["Hours.Per.Week"].values
x_test=test[list(features)].values

In [24]:
random.seed(100)
rf = RandomForestClassifier(n_estimators=1000)
#rf.fit(x_train, y_train)

## - Estimation of Performance:

In [25]:
"""
status = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, status[:,1])
roc_auc = auc(fpr, tpr)
print(roc_auc)

final_status = rf.predict_proba(x_test)
test["Hours.Per.Week"]=final_status[:,1]
test.to_csv('model_output.csv',columns=['ID','Hours.Per.Week'])
"""

'\nstatus = rf.predict_proba(x_validate)\nfpr, tpr, _ = roc_curve(y_validate, status[:,1])\nroc_auc = auc(fpr, tpr)\nprint(roc_auc)\n\nfinal_status = rf.predict_proba(x_test)\ntest["Hours.Per.Week"]=final_status[:,1]\ntest.to_csv(\'model_output.csv\',columns=[\'ID\',\'Hours.Per.Week\'])\n'